In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [ ]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

top_7_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(7)

In [ ]:
top_7_products.head(7)

In [ ]:
transactions_train['repeat_purchase'] = transactions_train.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

In [ ]:
filler_products = transactions_train[(transactions_train['repeat_purchase'] > 1) & (transactions_train['t_year'] == 2020)].\
groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(5)

In [ ]:
top_12_products = pd.concat([top_7_products, filler_products], axis = 0)[['article_id']].drop_duplicates()

In [ ]:
top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [ ]:
top_12_products

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)